In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rexpi
import scipy.sparse
import time

# approximate action of the matrix exponential operator

compute $\exp(tA )b$ for a matrix $A\in\mathbb{C}^{k\times k}$ which spectrum resides on the imaginary axis, a time-step $t$ and a vector $b\in\mathbb{C}^k$.

The time-step $t$ can be understood as the frequency $\omega$, i.e., $t=\omega$.

In [2]:
n = 40
tol=1e-6
tolequi = 1e-3
w = rexpi.west(n,tol)
print("for n=%d and tol=%g, our error estimate suggests w=%f"%(n,tol,w))

r, info = rexpi.brib(w = w, n = n, tolequi = tolequi, info=1)
print("... compute unitary best approximant")
print("used %d iterations, error = %.2e, deviation = %.2e\n"%(info['iterations'],info['err'],info['dev']))

print("... compute coefficients for product and partial fraction form")
a0, aj, sj = r.getpartialfractioncoef(sym=True)
poles = sj

for n=40 and tol=1e-06, our error estimate suggests w=101.765352
... compute unitary best approximant
used 10 iterations, error = 9.83e-07, deviation = 6.94e-05

... compute coefficients for product and partial fraction form


## Tridiagonal matrix, errors in norm

In [3]:
# with direct solver, small dimension
nrm = lambda x : np.linalg.norm(x)

print("use brib for n=%d, tol=%g, and w=%f"%(n,tol,w))

# define diagonal matrix with spectrum between -1 and 1
k=50
print("random starting vector b with \u2016b\u2016=1, dimension k=%d\n" %k)
u = np.random.rand(k)
u = u/nrm(u)
nrmb0 = 1

e1 = np.ones(k-1)
e = np.ones(k)
o = np.zeros(k)

# H is shifted Laplace operator with eigenvalues between -1 and 1
H = scipy.sparse.diags([0.5*e1,o,0.5*e1], [-1,0,1])

#reference solution
yref = scipy.sparse.linalg.expm_multiply(1j*w*H,u)

mvconjiH = lambda x : -1j*H.dot(x)
mvSaIiH = lambda s,x : (np.linalg.inv(1j*H.toarray() - s*np.eye(k))).dot(x)

yproductform = rexpi.evalr_product(mvconjiH, mvSaIiH, u, poles)
print("use poles: \u2016y-exp(iwA)b\u2016 = %.2e" % nrm(yref-yproductform))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(yproductform)-nrmb0))

yparfrac = rexpi.evalr_partialfraction(mvSaIiH, u, aj, sj)
print("use partial fraction: \u2016y-exp(iwA)b\u2016 = %.2e" % nrm(yref-yparfrac))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(yparfrac)-nrmb0))

#print("(error in l2 norm is bounded by scalar error since the spectrum of H is in [-1,1] and ||u||=1)")
#print("(computing r(iH)u via partial fraction is only accurate for r of small degree)")

use brib for n=40, tol=1e-06, and w=101.765352
random starting vector b with ‖b‖=1, dimension k=50

use poles: ‖y-exp(iwA)b‖ = 8.21e-07
error in norm, |‖y‖-‖b‖| = 5.55e-16

use partial fraction: ‖y-exp(iwA)b‖ = 8.21e-07
error in norm, |‖y‖-‖b‖| = 1.18e-11



### compare with Pade

In [4]:
npade = 70
rpade = rexpi.pade(npade)

# evaluate r(iwH), argument has to be re-scaled by w
poles2 = rpade.getpoles()

mvconjiHw = lambda x : -1j*w*H.dot(x)
mvSaIiHw = lambda s,x : (np.linalg.inv(1j*w*H.toarray() - s*np.eye(k))).dot(x)

ypadepoles = rexpi.evalr_product(mvconjiHw, mvSaIiHw, u, poles2)
ap0, apj, spj = rpade.getpartialfractioncoef()

ypadeparfrac = rexpi.evalr_partialfraction(mvSaIiHw, u, apj, spj)

print("approximate exponential with Pade, same problem as above")
print("use poles: \u2016y-exp(iwA)b\u2016 = %.2e" % nrm(yref-ypadepoles))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypadepoles)-nrmb0))
print("use partial fraction: \u2016y-exp(iwA)b\u2016 = %.2e" % nrm(yref-ypadeparfrac))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypadeparfrac)-nrmb0))

approximate exponential with Pade, same problem as above
use poles: ‖y-exp(iwA)b‖ = 3.40e-03
error in norm, |‖y‖-‖b‖| = 5.55e-16

use partial fraction: ‖y-exp(iwA)b‖ = 2.26e+15
error in norm, |‖y‖-‖b‖| = 2.26e+15



### compare with polynomial Chebyshev

In [5]:
ncheb = 130
# evaluate Chebyshev approximation using Clenshaw Algorithm

# evaluate with mv using Clenshaw Algorithm
mvH = lambda x : H.dot(x)
ypcheb = rexpi.chebyshev(mvH,w,u,ncheb)

print("approximate exponential with polynomial Chebyshev, same problem as above")
print("error: \u2016y-exp(iwA)b\u2016 = %.2e" % nrm(yref-ypcheb))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypcheb)-nrmb0))

approximate exponential with polynomial Chebyshev, same problem as above
error: ‖y-exp(iwA)b‖ = 1.69e-08
error in norm, |‖y‖-‖b‖| = 5.54e-09



## use unitary best approximant with Lapack tridiagonal solver
large dimensional problem

In [6]:
# with direct solver, small dimension
nrm = lambda x : np.linalg.norm(x)
print("use brib for n=%d, tol=%g, and w=%f"%(n,tol,w))
# define diagonal matrix with spectrum between -1 and 1
k=30000
print("random starting vector b with \u2016b\u2016=1, dimension k=%d\n" %k)
u = np.random.rand(k)
u = u/nrm(u)

e1 = np.ones(k-1,dtype=np.cdouble)
e = np.ones(k,dtype=np.cdouble)
o = np.zeros(k,dtype=np.cdouble)

# H is shifted Laplace operator with eigenvalues between -1 and 1
H = scipy.sparse.diags([0.5*e1,o,0.5*e1], [-1,0,1])

t1=time.time()
yref = scipy.sparse.linalg.expm_multiply(1j*w*H,u)
dt=time.time()-t1
print("time to compute reference solution %f"%dt)

use brib for n=40, tol=1e-06, and w=101.765352
random starting vector b with ‖b‖=1, dimension k=30000

time to compute reference solution 0.551987


In [7]:
mvconj = lambda x : -1j*H.dot(x)

# deep copies
dl2 = 1j*0.5*e1
dd2 = 1j*o
du2 = 1j*0.5*e1
def mvsHzgtsv(pole,b):
    [_, _, _, x, info]=scipy.linalg.lapack.zgtsv(dl2, dd2 - pole*e, du2, b)
    return x

In [8]:
t1=time.time()
ypadepoles = rexpi.evalr_product(mvconj, mvsHzgtsv, u, poles)
dt=time.time()-t1
print("time to evaluate approximation in product form %f"%dt)
print("use partial fraction: \u2016r(ix)-exp(iwx)\u2016 = %.2e" % nrm(yref-ypadepoles))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypadepoles)-nrmb0))

t1=time.time()
yparfrac = rexpi.evalr_partialfraction(mvsHzgtsv, u, aj, sj)
dt=time.time()-t1
print("time to evaluate approximation in partial fraction form %f"%dt)
#print("max |aj| = %.2e, min Re(sj) = %.2e" % (max(np.abs(aj)),min(sj.real)))
print("use partial fraction: \u2016r(ix)-exp(iwx)\u2016 = %.2e" % nrm(yref-yparfrac))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(yparfrac)-nrmb0))

print("use product vs partial fraction: \u2016r1(ix)-r2(ix)\u2016 = %.2e" % nrm(ypadepoles-yparfrac))

time to evaluate approximation in product form 0.100731
use partial fraction: ‖r(ix)-exp(iwx)‖ = 9.19e-07
error in norm, |‖y‖-‖b‖| = 4.44e-16

time to evaluate approximation in partial fraction form 0.067838
use partial fraction: ‖r(ix)-exp(iwx)‖ = 9.19e-07
error in norm, |‖y‖-‖b‖| = 1.02e-11

use product vs partial fraction: ‖r1(ix)-r2(ix)‖ = 4.06e-11
